# 取得台股日線資料

到台灣證券交易所取得資料
[證交所](https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201119&type=ALLBUT0999)

In [3]:
import pandas as pd
import requests
import io

In [4]:
url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20201119&type=ALLBUT0999'
page = requests.get(url)

In [5]:
pd.read_csv(io.StringIO('1,2,3 \n 1,2,3'))

,1,2,3
0,1,2,3


## 1. 將文字檔處理成pandas可以處理的格式

In [6]:
use_text = page.text.splitlines()
use_text[205] ## 作弊直接從excel看他在第幾行

'"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",'

### 目標是抓從證券代號開始的表格 所以搜尋他是在第幾行

* 作弊直接從excel看
* 用迴圈搜尋

In [7]:
for i, data in enumerate([2,4,6]):
    print(i, data)

0 2
1 4
2 6


In [8]:
for i, text in enumerate(use_text):
    if text.startswith('"證券代號","證券名稱","成交股數","成交筆數"') == 1:
        initial_point = i
        break

In [9]:
joinstr = ''.join([text[:-1] + '\n' for text in use_text[initial_point:]]) 
## 取代掉每個元素結尾的逗號換成換行符號 在把所有字串連接在一起
test_df = pd.read_csv(io.StringIO(joinstr))
test_df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,"=""0050""",元大台灣50,"4,505,205","3,786","514,299,847",114.45,114.55,113.80,114.15,-,0.20,114.10,18,114.15,109,0.00
1,"=""0051""",元大中型100,"54,075",42,"2,237,445",41.26,41.45,41.26,41.42,+,0.16,41.40,2,41.41,2,0.00
2,"=""0052""",富邦科技,"1,106,750",342,"111,937,331",102.00,102.00,100.90,101.05,-,0.65,101.00,12,101.05,9,0.00
3,"=""0053""",元大電子,"34,197",19,"1,846,611",54.20,54.30,53.95,54.05,-,0.50,54.05,1,54.40,1,0.00
4,"=""0054""",元大台商50,"10,000",7,"267,310",26.70,26.80,26.70,26.80,+,0.10,26.78,50,26.79,1,0.00


In [15]:
test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
test_df.head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"4,505,205","3,786","514,299,847",114.45,114.55,113.80,114.15,-,0.20,114.10,18,114.15,109,0.00
1,0051,元大中型100,"54,075",42,"2,237,445",41.26,41.45,41.26,41.42,+,0.16,41.40,2,41.41,2,0.00
2,0052,富邦科技,"1,106,750",342,"111,937,331",102.00,102.00,100.90,101.05,-,0.65,101.00,12,101.05,9,0.00
3,0053,元大電子,"34,197",19,"1,846,611",54.20,54.30,53.95,54.05,-,0.50,54.05,1,54.40,1,0.00
4,0054,元大台商50,"10,000",7,"267,310",26.70,26.80,26.70,26.80,+,0.10,26.78,50,26.79,1,0.00


## 2.將以上內容包裝成一個function 輸入多個日期回傳多個df

In [18]:
def crawler(data_time):
    page_url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + data_time + "&type=ALLBUT0999"
    page = requests.get(page_url)
    use_text = page.text.splitlines()
    for i, text in enumerate(use_text):
        if text.startswith('"證券代號","證券名稱","成交股數","成交筆數"') == 1:
            initial_point = i
            break
    joinstr = ''.join([text[:-1] + '\n' for text in use_text[initial_point:]]) 
    ## 取代掉每個元素結尾的逗號換成換行符號 在把所有字串連接在一起
    test_df = pd.read_csv(io.StringIO(joinstr))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
    return test_df
            

In [20]:
crawler('20201123').head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"5,800,114","4,322","668,881,804",114.65,115.85,114.65,115.45,+,1.45,115.45,32,115.50,17,0.00
1,0051,元大中型100,"81,966",47,"3,343,035",40.87,40.94,40.63,40.73,+,0.35,40.75,3,40.80,3,0.00
2,0052,富邦科技,"1,260,375",424,"128,924,449",101.35,102.70,101.35,102.55,+,1.65,102.50,4,102.55,2,0.00
3,0053,元大電子,"37,532",32,"2,026,361",53.30,54.40,53.30,54.40,+,1.15,53.80,1,54.40,3,0.00
4,0054,元大台商50,"12,200",8,"320,627",26.19,26.35,26.19,26.30,+,0.14,26.30,49,26.41,1,0.00


## 3. 利用 datatime 物件操作加一天減一天

In [21]:
import datetime
import time

In [22]:
date = datetime.datetime.now()
date

datetime.datetime(2020, 11, 23, 21, 21, 49, 889420)

In [23]:
date = str(date).split(' ')[0]
date

'2020-11-23'

In [24]:
date = ''.join(date.split('-'))
date

'20201123'

In [25]:
def trans_date(date_time):
    return ''.join(str(date_time).split(' ')[0].split('-'))

In [26]:
def parse_n_days(start_date, n):
    df_dict = {}
    now_date = start_date
    for i in range(n):
        time.sleep(3)
        now_date = now_date - datetime.timedelta(days = 1)
        try:
            df = crawler(trans_date(now_date))
            print("Success!" + ' ' + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
        except:
            print('Fails at' + ' ' + trans_date(now_date))
    return df_dict           

In [27]:
result_dict = parse_n_days(datetime.datetime.now(), 6)

Fails at 20201122
Fails at 20201121
Success! 20201120
Success! 20201119
Success! 20201118
Success! 20201117


In [28]:
result_dict.keys()

dict_keys(['20201120', '20201119', '20201118', '20201117'])

In [32]:
for key in result_dict.keys():
    result_dict[key].to_csv(str(key)+'.csv', encoding = "utf-8-sig")